# Все необходимые пакеты для работы


In [ ]:
# Чистка текстов построение корпуса
!pip install -U spacy
!pip install alphabet-detector
!python -m spacy download zh_core_web_lg

Очистка текста -> построение корпуса. В китайском языке отсутствует необхожимость в лематизации. Токенезирую с помощью Spacy. Фильтрую по частям речи, т. е. избавляюсь от служебных частиц, предлогов, союзов, знаков пунктуации и т. д. см banned_types). Также избавляюсь от некитайских символов (японские слова и латиница). Заменяю числа на абстрактное слово "Число", местоимения и имена собственные на абстрактное слово "Объект".

In [ ]:
# Libraries
import spacy
from spacy import displacy
from alphabet_detector import AlphabetDetector

banned_types = {
    "ADP", "AUX", "CONJ", "CCONJ", "DET", "INTJ", "PART", "PUNCT", "SYM", "X"
}

ad = AlphabetDetector()

# Funcs
def prepare_chinese(input_path):
    file_name = input_path.split('/')[-1]
    try:
        with open(input_path, encoding='utf-8') as lit_text:
            raw_text_lines = lit_text.readlines()
    except Exception as ex:
        return
    prepared_text = ""
    for nlp_doc in nlp.pipe(raw_text_lines):
        for token in nlp_doc:
            if not "LATIN" in ad.detect_alphabet(token.text):
                if (token.pos_ == "PRON" or token.pos_ == "PROPN"):
                    prepared_text += "物体"
                elif token.pos_ == "NUM":
                    prepared_text += "数字"
                elif not token.pos_ in banned_types:
                    prepared_text += token.text
    return prepared_text

def add_to_corpus(text, out_file):
    src_list = text.split()
    text = "".join(src_list)
    text = re.sub(r'[^\u4e00-\u9fff]+', '',text)
    
    out_file.write(text)
    out_file.write('\n')

Дополнительная очистка корпуса (если понадобится пока не испльзовал). Мотивация применения в том что, на случай, если модель Spacy не обнаружила все имена собственные, то из можно обнаружить и заменить на общее слово ассоциирующее с ними.

In [ ]:
import spacy
from spacy import displacy

path = 'corpus_ch.txt'
new_path = 'clean_corpus_ch.txt'
nlp = spacy.load("zh_core_web_lg", disable=['tagger', 'parser'])
nlp.max_length = 17000000
with open(path, 'r', encoding='utf-8') as corpus:
    with open(new_path, 'w', encoding='utf-8') as cleaned_corpus:
        texts = corpus.readlines()
        for text_line in texts:
            nlp_doc = nlp(text_line)
            for ent in nlp_doc.ents:
                text_line = text_line.replace(str(ent.text), str(ent.label_) + '_', 1)
            cleaned_corpus.write(text_line)
            cleaned_corpus.write('\n')


# Построение словаря

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba
from scipy.sparse.linalg import svds
import numpy as np

Основные стоп слова - которые токенайзер будет фильтровать, на случай, если при очистке корпуса они не отфильтровались

In [ ]:
chinese_stopwords = ['"','0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0','!', '@', '#', '$', '^', '&', '*', '—','’','‘',']','[','_','·','.%',')','(','”','“','\u3000',"、","。","〈","〉","《","》","一","一些","一何","一切","一则","一方面","一旦","一来","一样","一般",
                     "一一", "一转眼","七","万一","三","上","上下","下","不","不仅","不但","不光","不单","不只","不外乎","不如",
                     "不妨","不尽","不尽然","不得","不怕","不惟","不成","不拘","不料","不是","不比","不然","不特","不独",
                     "不管","不至于","不若","不论","不过","不问","与","与其","与其说","与否","与此同时","且","且不说","且说",
                     "两者","个","个别","中","临","为","为了","为什么","为何","为止","为此","为着","乃","乃至","乃至于","么",
                     "之","之一","之所以","之类","乌乎","乎","乘","九","也","也好","也罢","了","二","二来","于","于是",
                     "于是乎","云云","云尔","五","些","亦","人","人们","人家","什","什么","什么样","今","介于","仍","仍旧",
                     "从","从此","从而","他","他人","他们","他们们","以","以上","以为","以便","以免","以及","以故","以期","以来",
                     "以至","以至于","以致","们","任","任何","任凭","会","似的","但","但凡","但是","何","何以","何况","何处","何时",
                     "余外","作为","你","你们","使","使得","例如","依","依据","依照","便于","俺","俺们","倘","倘使","倘或","倘然",
                     "倘若","借","借傥然","假使","假如","假若","做","像","儿","先不先","光是","全体","全部","八","六","兮","共",
                     "关于","关于具体地说","其","其一","其中","其二","其他","其余","其它","其次","具体地说","具体说来","兼之","内",
                     "再","再其次","再则","再有","再者","再者说","再说","冒","冲","况且","几","几时","凡","凡是","凭","凭借","出于",
                     "出来","分","分别","则","则甚","别","别人","别处","别是","别的","别管","别说","到","前后","前此","前者","加之",
                     "加以","即","即令","即使","即便","即如","即或","即若","却","去","又","又及","及","及其","及至","反之","反而",
                     "反过来","反过来说","受到","另","另一方面","另外","另悉","只","只当","只怕","只是","只有","只消","只要","只限",
                     "叫","叮咚","可","可以","可是","可见","各","各个","各位","各种","各自","同","同时","后","后者","向","向使",
                     "向着","吓","吗","否则","吧","吧哒","含","吱","呀","呃","呕","呗","呜","呜呼","呢","呵","呵呵","呸","呼哧",
                     "咋","和","咚","咦","咧","咱","咱们","咳","哇","哈","哈哈","哉","哎","哎呀","哎哟","哗","哟","哦","哩",
                     "哪","哪个","哪些","哪儿","哪天","哪年","哪怕","哪样","哪边","哪里","哼","哼唷","唉","唯有","啊","啐","啥",
                     "啦","啪达","啷当","喂","喏","喔唷","喽","嗡","嗡嗡","嗬","嗯","嗳","嘎","嘎登","嘘","嘛","嘻","嘿","嘿嘿",
                     "四","因","因为","因了","因此","因着","因而","固然","在","在下","在于","地","基于","处在","多","多么","多少",
                     "大","大家","她","她们","好","如","如上","如上所述","如下","如何","如其","如同","如是","如果","如此","如若",
                     "始而","孰料","孰知","宁","宁可","宁愿","宁肯","它","它们","对","对于","对待","对方","对比","将","小","尔",
                     "尔后","尔尔","尚且","就","就是","就是了","就是说","就算","就要","尽","尽管","尽管如此","岂但","己","已",
                     "已矣","巴","巴巴","年","并","并且","庶乎","庶几","开外","开始","归","归齐","当","当地","当然","当着","彼",
                     "彼时","彼此","往","待","很","得","得了","怎","怎么","怎么办","怎么样","怎奈","怎样","总之","总的来看",
                     "总的来说","总的说来","总而言之","恰恰相反","您","惟其","慢说","我","我们","或","或则","或是","或曰","或者",
                     "截至","所","所以","所在","所幸","所有","才","才能","打","打从","把","抑或","拿","按","按照","换句话说",
                     "换言之","据","据此","接着","故","故此","故而","旁人","无","无宁","无论","既","既往","既是","既然","日","时",
                     "时候","是","是以","是的","更","曾","替","替代","最","月","有","有些","有关","有及","有时","有的","望","朝",
                     "朝着","本","本人","本地","本着","本身","来","来着","来自","来说","极了","果然","果真","某","某个","某些",
                     "某某","根据","欤","正值","正如","正巧","正是","此","此地","此处","此外","此时","此次","此间","毋宁","每",
                     "每当","比","比及","比如","比方","没奈何","沿","沿着","漫说","焉","然则","然后","然而","照","照着","犹且",
                     "犹自","甚且","甚么","甚或","甚而","甚至","甚至于","用","用来","由","由于","由是","由此","由此可见","的",
                     "的确","的话","直到","相对而言","省得","看","眨眼","着","着呢","矣","矣乎","矣哉","离","秒","竟而","第",
                     "等","等到","等等","简言之","管","类如","紧接着","纵","纵令","纵使","纵然","经","经过","结果","给","继之",
                     "继后","继而","综上所述","罢了","者","而","而且","而况","而后","而外","而已","而是","而言","能","能否",
                     "腾","自","自个儿","自从","自各儿","自后","自家","自己","自打","自身","至","至于","至今","至若","致","般的",
                     "若","若夫","若是","若果","若非","莫不然","莫如","莫若","虽","虽则","虽然","虽说","被","要","要不","要不是",
                     "要不然","要么","要是","譬喻","譬如","让","许多","论","设使","设或","设若","诚如","诚然","该","说","说来",
                     "请","诸","诸位","诸如","谁","谁人","谁料","谁知","贼死","赖以","赶","起","起见","趁","趁着","越是","距","跟",
                     "较","较之","边","过","还","还是","还有","还要","这","这一来","这个","这么","这么些","这么样","这么点儿","这些",
                     "这会儿","这儿","这就是说","这时","这样","这次","这般","这边","这里","进而","连","连同","逐步","通过","遵循","遵照",
                     "那","那个","那么","那么些","那么样","那些","那会儿","那儿","那时","那样","那般","那边","那里","都","鄙人","鉴于","针对",
                     "阿","除","除了","除外","除开","除此之外","除非","随","随后","随时","随着","难道说","零","非","非但","非徒","非特","非独",
                     "靠","顺","顺着","首先","︿","！","＃","＄","％","＆","（","）","＊","＋","，","０","１","２","３","４","５","６","７","８",
                     "９","：","；","＜","＞","？","＠","［","］","｛","｜","｝","～","￥", ""]

Построение словаря как здесь https://colab.research.google.com/drive/1uRlhcPKqaTfOd6J9pxs9yrv5lbaDFClQ, в качестве токенайзера использую jieba, для нарезки он лучше чем spacy

In [ ]:
def make_table_and_dict(corpus_path, min_df, max_df):
    with open(corpus_path, 'r', encoding = 'utf-8') as corpus_file:
        vectorizer = TfidfVectorizer(analyzer = 'word', min_df = min_df, max_df = max_df, stop_words= chinese_stopwords, tokenizer= jieba.lcut)
        data_vectorized = vectorizer.fit_transform(corpus_file)
    return data_vectorized, vectorizer.get_feature_names(), vectorizer.idf_

def create_table(data_vectorized, k, table_path, name):
    u, sigma, vt = svds(data_vectorized, k)
    print(sigma)
    with open(name + str(k) + '.npy', 'wb') as f:
        np.save(f, np.dot(np.diag(sigma), vt).T)


In [ ]:
path = 'corpus_ch.txt'
zh_data_vectorized, zh_dictionary, idfs = make_table_and_dict(path, 0.01, 0.8)
with open('words_v4.npy', 'wb') as f:
    np.save(f, zh_dictionary)
print(len(zh_dictionary))
create_table(zh_data_vectorized, 1024, path, "zh_sigma_v4_")

Получаю словарь, представляемый в виде двух файлов: words_v.npy это столбец уникальных лексем словаря, а zh_sigma_v_k.npy соответствуемый слову вектор. Ниже функция, дающая классический питоновский словарь из такого представления

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np

words = np.load('/content/gdrive/My Drive/words_v3.npy')
vectors = np.load('/content/gdrive/My Drive/zh_sigma_v3_1024.npy')

In [ ]:
def build_dict(key_word, vectors):
  zipper = zip(key_word, vectors)
  return dict(zipper)

In [ ]:
chinese_dict = build_dict(words, vectors)

In [ ]:
print(chinese_dict['黄瓜'])

[ 0.00171133 -0.00393914 -0.00495353 ... -0.00192966 -0.00033586
 -0.01994419]
